In [12]:
import argparse
import pandas as pd
import os
from tqdm import tqdm

def run_convert(input_folder, output_folder = None):

    # Create storage
    gene_annotations = None
    gene_counts_tpm = []
#     gene_counts_fpkm = []
#     gene_counts_fpkm_uq = []

    # Load patients list
    df_patients = None
    for file in os.listdir(input_folder):
        if file.split('.')[0] == "gdc_sample_sheet":
            df_patients = pd.read_csv(os.path.join(input_folder, file), sep="\t").set_index("File ID")
            break
    assert df_patients is not None, "'gdc_sample_sheet.tsv' is missing !"

    # Load files list
    rna_folder_path  = os.path.join(input_folder, 'RNA-seq')
    file_list = os.listdir(rna_folder_path)

    # Create output matrices
    for case_file in tqdm(df_patients.index, total=len(df_patients.index)):
        if case_file in file_list:
            case_path = os.path.join(rna_folder_path, case_file)
            file_path = None
            for f in os.listdir(case_path):
                if f.endswith(".tsv"):
                    file_path = os.path.join(case_path, f)
            if file_path is not None:
                df_rnaseq = pd.read_csv(file_path, sep="\t", skiprows=1).iloc[4:, :].set_index("gene_id")
                counts_tpm = pd.Series({"Case ID": df_patients.loc[case_file, "Case ID"] ,
                                        "Sample Type": df_patients.loc[case_file, "Sample Type"]})
#                 counts_fpkm = counts_tpm.copy()
#                 counts_fpkm_uq = counts_tpm.copy()

                counts_tpm = pd.concat([counts_tpm, df_rnaseq["tpm_unstranded"]])
#                 counts_fpkm = pd.concat([counts_fpkm, df_rnaseq["fpkm_unstranded"]])
#                 counts_fpkm_uq = pd.concat([counts_fpkm_uq, df_rnaseq["fpkm_uq_unstranded"]])

                gene_counts_tpm.append(counts_tpm.rename(df_patients.loc[case_file, "Sample ID"]))
#                 gene_counts_fpkm.append(counts_fpkm.rename(df_patients.loc[case_file, "Sample ID"]))
#                 gene_counts_fpkm_uq.append(counts_fpkm_uq.rename(df_patients.loc[case_file, "Sample ID"]))
            else:
                print(
                    "RNA-seq data of sample " + df_patients.loc[ind, "Sample ID"] + " are missing ! Check file " + ind)
        else:
            print("RNA-seq data of sample " + df_patients.loc[ind, "Sample ID"] + " are missing ! Check file " + ind)

    gene_counts_tpm = pd.concat(gene_counts_tpm, axis=1)
#     gene_counts_fpkm = pd.concat(gene_counts_fpkm, axis=1)
#     gene_counts_fpkm_uq = pd.concat(gene_counts_fpkm_uq, axis=1)

    # Create annotation matrix
    assert file_path is not None, "No rnaseq file was found, check your folder architecture !"
    gene_annotations = df_rnaseq[['gene_name', 'gene_type']]

    # Create output folder
    if output_folder is None:
        output_folder = os.path.join(input_folder, 'merged_RNAseq')
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    gene_counts_tpm.to_csv(os.path.join(output_folder, "tpm.csv"))
#     gene_counts_fpkm.to_csv(os.path.join(output_folder, "fpkm.csv"))
#     gene_counts_fpkm_uq.to_csv(os.path.join(output_folder, "fpkm_uq.csv"))
    gene_annotations.to_csv(os.path.join(output_folder, "gene_annotations.csv"))
    return


In [17]:
from glob import glob
for f in glob('/mnt/data/lrebaud/Documents/ICARE_TCGA/download/*'):
    if os.path.isfile(f+'/merged_RNAseq/tpm.csv'):
        continue
    print(f)
    run_convert(f)

/mnt/data/lrebaud/Documents/ICARE_TCGA/download/LGG


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 534/534 [03:13<00:00,  2.76it/s]


/mnt/data/lrebaud/Documents/ICARE_TCGA/download/PAAD


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 183/183 [01:00<00:00,  3.04it/s]


/mnt/data/lrebaud/Documents/ICARE_TCGA/download/LUAD


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 122/122 [00:45<00:00,  2.70it/s]


/mnt/data/lrebaud/Documents/ICARE_TCGA/download/SKCM


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 473/473 [02:46<00:00,  2.85it/s]


/mnt/data/lrebaud/Documents/ICARE_TCGA/download/KIRC


 53%|██████████████████████████████████████████████████████████████████████████▌                                                                 | 327/614 [01:57<01:47,  2.67it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
import shutil


for f in tqdm(glob('/mnt/data/lrebaud/Documents/ICARE_TCGA/download/*')):
    print(f)
    d = f.split('/')[-1]
    path_new = '/mnt/data/lrebaud/data/TCGA/RNA-seq/'+d+'/'
    try:
        os.mkdir(path_new)
    except:
        pass
    shutil.copyfile(f+'/merged_RNAseq/tpm.csv', path_new+'tpm.csv')
    shutil.copyfile(f+'/merged_RNAseq/gene_annotations.csv', path_new+'gene_annotations.csv')

  0%|                                                                                                                                                       | 0/33 [00:00<?, ?it/s]

/mnt/data/lrebaud/Documents/ICARE_TCGA/download/HNSC


  3%|████▎                                                                                                                                          | 1/33 [00:27<14:55, 27.99s/it]

/mnt/data/lrebaud/Documents/ICARE_TCGA/download/DLBC


  6%|████████▋                                                                                                                                      | 2/33 [00:30<06:47, 13.14s/it]

...

 94%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍        | 31/33 [08:37<00:42, 21.08s/it]

/mnt/data/lrebaud/Documents/ICARE_TCGA/download/CESC


 97%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋    | 32/33 [08:54<00:20, 20.02s/it]

/mnt/data/lrebaud/Documents/ICARE_TCGA/download/KICH


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [08:58<00:00, 16.31s/it]
